In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Input  

df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

X = df.drop(columns=['Class'])
y = df['Class']



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=1, shuffle=True)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [32]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Keep the loss suitable for binary classification
    metrics=[
        tf.keras.metrics.Recall(),  # Prioritize recall
        tf.keras.metrics.Precision(),  # Optionally track precision as well
        'accuracy'  # Still track accuracy for reference
    ]
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [33]:
# Train the model using the GPU (if available)
with tf.device('/GPU:0'):  # Automatically uses GPU if available
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping], verbose=2)


Epoch 1/100


W0000 00:00:1724252968.675859      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1724252978.249188      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5341/5341 - 14s - 3ms/step - accuracy: 0.9993 - loss: 0.0061 - precision_4: 0.8479 - recall_5: 0.7240 - val_accuracy: 0.9993 - val_loss: 0.0035 - val_precision_4: 0.8000 - val_recall_5: 0.7671
Epoch 2/100
5341/5341 - 11s - 2ms/step - accuracy: 0.9994 - loss: 0.0029 - precision_4: 0.8737 - recall_5: 0.8084 - val_accuracy: 0.9993 - val_loss: 0.0035 - val_precision_4: 0.8889 - val_recall_5: 0.6575
Epoch 3/100
5341/5341 - 10s - 2ms/step - accuracy: 0.9994 - loss: 0.0027 - precision_4: 0.8813 - recall_5: 0.7955 - val_accuracy: 0.9994 - val_loss: 0.0032 - val_precision_4: 0.9123 - val_recall_5: 0.7123
Epoch 4/100
5341/5341 - 10s - 2ms/step - accuracy: 0.9995 - loss: 0.0022 - precision_4: 0.9011 - recall_5: 0.7987 - val_accuracy: 0.9992 - val_loss: 0.0032 - val_precision_4: 0.7703 - val_recall_5: 0.7808
Epoch 5/100
5341/5341 - 10s - 2ms/step - accuracy: 0.9995 - loss: 0.0019 - precision_4: 0.9167 - recall_5: 0.8214 - val_accuracy: 0.9994 - val_loss: 0.0033 - val_precision_4: 0.8485 - val_reca

In [34]:
def print_scores(y_true, y_pred, model_name):
    print(f"{model_name} Precision: {precision_score(y_true, y_pred):0.2f}")
    print(f"{model_name} Recall: {recall_score(y_true, y_pred):0.2f}")
    print(f"{model_name} F1-Score: {f1_score(y_true, y_pred):0.2f}")

In [35]:
# thresholds = [i/10 for i in range(6)]
# for threshold in thresholds:
threshold = 0.4
y_test_pred = (model.predict(X_test) > threshold).astype(int).flatten()
y_train_pred = (model.predict(X_train) > threshold).astype(int).flatten()

print(f"Threshold = {threshold}")
print_scores(y_test, y_test_pred, 'Neural Network (Test)')
print_scores(y_train, y_train_pred, 'Neural Network (Train)')

 138/2226 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step

W0000 00:00:1724253062.913062      89 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2226/2226 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step
Threshold = 0.4
Neural Network (Test) Precision: 0.80
Neural Network (Test) Recall: 0.80
Neural Network (Test) F1-Score: 0.80
Neural Network (Train) Precision: 0.83
Neural Network (Train) Recall: 0.87
Neural Network (Train) F1-Score: 0.85


In [36]:
model.save('neural_network_model.h5')